## Interacting with the Batfish service


### Python Imports

In your Python program (or shell) you will need to import Pybatfish modules. 
The most common imports are shown below. Depending your needs, this list may vary.

In [1]:
import pandas as pd
from pybatfish.client.session import Session
from pybatfish.datamodel import *
from pybatfish.datamodel.answer import *
from pybatfish.datamodel.flow import *

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option('display.html.use_mathjax', False)

In [3]:
# Configure all pybatfish loggers to use WARN level
import logging
logging.getLogger('pybatfish').setLevel(logging.WARN)

### Sessions

The Batfish service may be running locally on your machine, or on a remote server. 
The first step to analyzing your configurations is setting up the connection to the Batfish service.

In [4]:
bf = Session(host="localhost")

### Uploading configurations

Batfish is designed to analyze a series of snapshots of a network.

A *network* is a logical grouping of devices -- it may mean all of the devices in your network, 
or a subset (e.g., all devices in a single datacenter.)
 
A *snapshot* is a state of the network at a given time.
A network may contain many snapshots, allowing you to understand the evolution of your network.

Let's say we will be working with our example datacenter:

In [5]:
bf.set_network('example_dc')

'example_dc'

Now you are ready to create your first snapshot. Batfish can ingest a variety of data in order to model your network,
so let's look at how you can package it as a snapshot.

#### Packaging snapshot data

Batfish expects snapshot data to be organized in a specific folder structure. 

***

* snapshot _[top-level folder]_
    * **configs** _[folder with configurations files of network devices]_
        * router1.cfg
        * router2.cfg
        * ...
    * **batfish** _[supplemental information (not device configurations)]_
        * isp_config.json
        * ...

*** 

See [this snapshot](https://github.com/batfish/batfish/tree/master/networks/example) for an example. For illustration, it contains some files that are not used by Batfish, e.g., `example-network.png` (network diagrams are not needed). It also contains information for host modeling, which need not be provided if you are not modeling hosts. 

When you supply the snapshot as a zipped file, the top-level folder (called "snapshot" above) should be part of the zip archive. 

Details on the format of configuration files and supplemental information [are described here](../formats.md)

#### Initializing a new snapshot

In [6]:
SNAPSHOT_DIR = '../../networks/example'
bf.init_snapshot(SNAPSHOT_DIR, name='snapshot-2020-01-01', overwrite=True)

'snapshot-2020-01-01'

### Using an existing snapshot

If you have a previously initialized snapshot that you would like to work with, you do not need to re-initialize it.
Simply set the network and snapshot by name:

In [7]:
bf.set_network('example_dc')
bf.set_snapshot('snapshot-2020-01-01')

'snapshot-2020-01-01'

### Running Questions

Now that you have initialized (or set) a snapshot, 
you can query the Batfish service to retrieve information about the snapshot.

Batfish exposes a series of *questions* to users.
With the help of these questions you can examine data about you network as a whole,
or individual devices, in a vendor-agnostic way.

Note that while Batfish supports a wide variety of devices and configuration constructs, 
it may not fully support your configuration files. 
We always recommend checking the status of the snapshot you just initialized, by runnning `bf.q.initIssues`:

In [8]:
bf.q.initIssues().answer()

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,['as1border1'],None,Convert warning (redflag),Could not determine update source for BGP neighbor: '5.6.7.8',None,None
1,['as1border1'],None,Convert warning (redflag),Could not determine update source for BGP neighbor: '3.2.2.2',None,None


This is the general pattern you will be using for many questions:

* `bf.q.<question_name>()` Creates a question (with parameters, if applicable).
* `bf.q.<question_name>().answer()` sends a query to Batfish service and returns the results of executing the question
* `bf.q.<question_name>().answer().frame()` converts the answer into a [Pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe) for easy data manipulation

These are the very basics of how to interact with the Batfish service.

As a next step, explore [a variety of questions](../questions.rst) that allow you to analyze your network in great detail.

### Logging

The default logging of pybatfish is verbose to inform new users about what is happening. To control logging verbosity, use the following snippet toward the top of your Python script. Replace `logging.WARN` with your preferred [logging level](https://docs.python.org/3/library/logging.html#levels). 

In [9]:
import logging
logging.getLogger("pybatfish").setLevel(logging.WARN)